In [47]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain
import chromadb
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import YoutubeLoader
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

In [2]:
os.environ["GROQ_API_KEY"] = "key"

In [3]:

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [4]:
response = llm.invoke("Hey there ")
response.content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [10]:
load_dotenv()
key = os.getenv('GOOGLE_API_KEY')
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)


In [7]:
with open("Zukerber laama3-405b.txt") as f:
    text = f.read()

In [8]:
final_text = text_splitter.create_documents([text])
final_text

[Document(page_content='# Mark Zuckerberg on Llama 3.1, Open Source, AI Agents, Safety, and more\n# https://www.youtube.com/watch/Vy3OkbtUa5k'),
 Document(page_content="00:00:00.080 the first time we're releasing a 405\n00:00:02.280 billion parameter model by far the most\n00:00:04.279 sophisticated open source model that\n00:00:06.080 that I think anyone has has put out I\n00:00:07.600 was a little shocked by how directly you\n00:00:09.679 called out apple and their closed\n00:00:11.320 approach can you kind of expand on that\n00:00:13.839 and where Apple has been a blocker for\n00:00:15.839 meta yeah I think it's a little bit Soul"),
 Document(page_content="00:00:18.080 crushing when you go build features that\n00:00:20.519 are what you believe is good for for\n00:00:22.480 your community and then you're told that\n00:00:24.240 you can't ship them because some company\n00:00:26.760 wants to put you in a box so that they\n00:00:28.480 they can better compete with you can\n00:00:30.119

In [35]:
from yt_transcript import get_transcript



translation_language = "de"
transcript = get_transcript("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
type(transcript[0].page_content)

str

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
docs = text_splitter.split_documents(transcript)
docs

[Document(metadata={'source': 'dQw4w9WgXcQ', 'title': 'Never Gonna Give You Up', 'description': 'Unknown', 'view_count': 1558272665, 'thumbnail_url': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hq720.jpg?sqp=-oaymwEXCNUGEOADIAQqCwjVARCqCBh4INgESFo&rs=AOn4CLBX-HcaMSEAucUr5J0qD5nEyiPAoQ', 'publish_date': '2009-10-24 00:00:00', 'length': 212, 'author': 'Rick Astley'}, page_content="[Music]  we're no strangers to  love you know the rules and so do  I I full commitments while I'm thinking  of  you wouldn't get this from any other guy  I just want to tell you how I'm  feeling got to make you understand Never  Going To Give You Up never going to let  you down never going to run around and  desert you never going to make you cry  never going to say goodbye never going  to tell a lie and hurt you  we've known each other for so  long your heart's been aching but your  to sh to"),
 Document(metadata={'source': 'dQw4w9WgXcQ', 'title': 'Never Gonna Give You Up', 'description': 'Unknown', 'view_count': 1558

In [38]:
persistent_dir = "chroma_db"
db = Chroma.from_documents(documents=docs,embedding=embedding_function,persist_directory=persistent_dir)
db.persist()

In [39]:
read_db = Chroma(persist_directory=persistent_dir,embedding_function=embedding_function)

In [53]:
db.similarity_search("what is rickroll")
retriever = db.as_retriever()

In [43]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=False)

user_input = ""
while user_input.strip() != "/bye":
    user_input = input('\nHuman: ').strip() 
    print(f"\nHuman : {user_input}")
    if user_input == "/bye":
        break  
    matching_docs = db.similarity_search(user_input)
    output = chain.run(input_documents=matching_docs,question=user_input)
    print(f'\nAI: {output}')
    print('')



Human : hello

AI: Hello! It seems like we have some song lyrics here, specifically from "Never Gonna Give You Up" by Rick Astley. What would you like to know or talk about?


Human : tell mw what is singer talking about

AI: The singer appears to be talking about commitment and devotion to someone they care about. They are promising to never let this person down, never desert them, never make them cry, and never say goodbye. The repetition of the phrase "Never Going to Give You Up" suggests a strong promise to stand by this person and maintain a relationship with them. The overall tone is one of reassurance and loyalty.


Human : /bye


In [50]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question
if you dont know the answer just tell that you dont know the answer in very funny way:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [64]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=False,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"),
    }
)

In [65]:
user_input = ""
while user_input.strip() != "/bye":
    user_input = input('\nHuman: ').strip() 
    print(f"\nHuman : {user_input}")
    if user_input == "/bye":
        break  
    output = qa.run(user_input)
    print(f'\nAI: {output}')
    print('')



Human : hello there


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
you down never  going to run around and desert you never  going to make you going to  [Music]  goodbye  and

you down never  going to run around and desert you never  going to make you going to  [Music]  goodbye  and

make you understand Never  Going To Give You Up never going to let  you down never going to run around and  desert you never going to make you cry  never going to say goodbye never going  to tell you my and Hurt You Never Going  To Give You Up  never going to let you down never going  to run around and desert you never going  to make you C never going to say goodbye  never going to  tell and Hur You Never Going To Give You  Up never going to let you down never  going to run a